In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv as csv
import random
import math
from collections import Counter
eps = np.finfo(float).eps
from numpy import log2 as log
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
random.seed(42)

/kaggle/input/uofu-ml-fall-2020/project_data/sample-submissions/all-positive.csv
/kaggle/input/uofu-ml-fall-2020/project_data/sample-submissions/all-negative.csv
/kaggle/input/uofu-ml-fall-2020/project_data/data/eval.ids
/kaggle/input/uofu-ml-fall-2020/project_data/data/misc-attributes/misc-attributes-test.csv
/kaggle/input/uofu-ml-fall-2020/project_data/data/misc-attributes/misc-attributes-eval.csv
/kaggle/input/uofu-ml-fall-2020/project_data/data/misc-attributes/misc-attributes-train.csv
/kaggle/input/uofu-ml-fall-2020/project_data/data/tfidf/tfidf.train.libsvm
/kaggle/input/uofu-ml-fall-2020/project_data/data/tfidf/tfidf.test.libsvm
/kaggle/input/uofu-ml-fall-2020/project_data/data/tfidf/tfidf.eval.anon.libsvm
/kaggle/input/uofu-ml-fall-2020/project_data/data/glove/glove.eval.anon.libsvm
/kaggle/input/uofu-ml-fall-2020/project_data/data/glove/glove.test.libsvm
/kaggle/input/uofu-ml-fall-2020/project_data/data/glove/glove.train.libsvm
/kaggle/input/uofu-ml-fall-2020/project_data/data

In [2]:
def newTable(data):
    max = 0
    count = 0
    
    with open(data, 'r', newline ='') as f:
        reader = csv.reader(f, delimiter=' ')
        for line in reader:
            count +=1
            
            for i in line:
                index = int(i.split(":")[0])
                
                if index > max:
                    max = index
    return np.zeros((count, max+1))

In [3]:
def fillTable(table,df):
    with open(df, 'r') as f:
        reader = csv.reader(f, delimiter=" ")
        row = -1
        for i in reader:
            row +=1
            
            for o in i:
                data = o.split(":")
                
                if(int(data[0]) == 0 or int(data[0])==1):
                    table[row, 0] = int(data[0])
                else:
                    table[row, int(data[0])] = int(data[1])

    return table

In [4]:
def createNPY(input):
    print(f"{input}")
    table = newTable(input)
    fillTable(table,input)
    print("Done")
    return table

In [5]:
def accuracy(labels, predictions):
  correct = 0

  for i in range(len(labels)):
    if labels[i] == predictions[i]:
      correct += 1

  return correct / float(len(labels)) * 100.0

**Decision Tree**

In [6]:
def infoGain(data, p):
    values = []
    
    for i in range(1,data.shape[1]):
        ig = p - entropy(data,i)
        values.append(ig)
    
    value = max(values)
    index = values.index(value)

    return index, value

In [7]:
def totalEntropy(data):
    entropy = 0
    length = data.shape[0]
    
    for i in range(data.shape[1]):
        pos = 0
        for j in range(length):
            if data[j, i] > 0:
                pos += 1
        
        p = pos / length
        entropy += -p*log(p+eps)
    
    return entropy / length

In [8]:
def entropy(data, feature):
    features = np.unique(data[:,feature])
    p = 0
    
    for i in features:
        p_i=0
        pos=0
        num = 0
        
        for j in range(data.shape[0]):
            if data[j, feature] == i and data[j,0] == 1:
                pos +=1
            
            if(data[j, feature] == i):
                num += 1
                
            p_1 = pos/(num+eps)
            p_i += -p_1+log(p_1+eps)
            
        p_2 = num / len(data)
        p += -p_2*p_i
    
    return abs(p)

In [9]:
def buildTree(data, constantData, depth, totalEntropy, currentDepth=0, parent=None):
    
    feature, ig = infoGain(data, totalEntropy)
    
    #label check
    if len(np.unique(data[:,0])) <= 1:
        return np.unique(data[:,0])[0]
    
    elif len(data) == 0:
        return grabNode(constantData)
    elif currentDepth >= depth:
        return grabNode(data)
    else:
        parent = grabNode(data)
        
        #which feature to spli on
        index, ig = infoGain(data, totalEntropy)
        
        tree = {index:{}}
        
        features = np.unique(data[:,index])
        for i in features:
            child = split(data, index, i)
            
            subtree = buildTree(child, constantData, depth, totalEntropy, currentDepth+1, parent)
            
            tree[index][i] = subtree
            
    return tree

In [10]:
def grabNode(dataset):
    return np.unique(dataset[:,0])[np.argmax(np.unique(dataset[:,0],return_counts=True)[1])]

In [11]:
def split(data, feature, value):
    examples = []
    for i in range(data.shape[0]):
        
        if data[i, feature] == value:
            examples.append(data[i])
    
    arr = np.array(examples)
    
    return np.delete(arr, feature, 1)

In [12]:
def predict(labels,tree):
    for nodes in tree.keys():
        value = labels[nodes]
        tree = tree[nodes][value]
        prediction = 0
        if type(tree) is dict:
            prediction = tree_predict(labels, tree)
        else:
            prediction = tree
            break;
    return prediction

**Testing**

In [13]:
input_file_train = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.train.libsvm'
input_file_test = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.test.libsvm'
input_file_eval = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.eval.anon.libsvm'

train = createNPY(input_file_train)
test = createNPY(input_file_test)
evalTable = createNPY(input_file_eval)

train = train[:,:10000]
test = test[:,:10000]
evalTable = evalTable[:,:10000]




# pd.DataFrame(train).to_csv("/kaggle/working/train.csv", index=None)
# pd.DataFrame(test).to_csv("/kaggle/working/test.csv", index=None)
# pd.DataFrame(evalTable).to_csv("/kaggle/working/evalTable.csv", index=None)

/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.train.libsvm
Done
/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.test.libsvm
Done
/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.eval.anon.libsvm
Done


In [14]:
splitting = int(train.shape[0]/4)
print(splitting)

# print(train.shape[0])


train = train[:splitting,:]
print(train)
print(train.shape)
te = totalEntropy(train)
tree = buildTree(train, train, 100, te)

4375
[[1. 0. 2. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(4375, 10000)


In [15]:
print(tree)
features = test[:,1:]
labels = test[:,0:1]

predictions = []

for i in features:
    print(i)
    pred = predict(i,tree)
    print(pred)
    predictions.append(pred)



acc = accuracy(labels,predictions)
print(acc)

{0: {0.0: 0.0, 1.0: 0.0}}
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 2. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 6. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.

[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 6. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 2. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. ... 0. 0. 0.]
0.0
[0. 3. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 6. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[ 0.  0. 49. ...  0.  0.  0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. 

In [16]:
predictions = []
features = evalTable[:,1:]
for i in features:
    #print(i)
    pred = predict(i,tree)
    #print(pred)
    predictions.append(pred)



[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 7. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 5. ... 0. 0. 0.]
0.0
[0. 1. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 8. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 2. 2. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

[0. 1. 5. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 6. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 7. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 4. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 5. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 3. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[ 0.  0. 15. ...  0.  0.  0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. 

...

0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 4. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 1. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0.

0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 2. 1. ... 0. 0. 0.]
0.0
[0. 0. 4. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0.

[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 2. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 9. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 2. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 

0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 4. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[ 0.  0. 18. ...  0.  0.  0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 6. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 1. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 0. ... 0. 0. 0.]
0.0
[0. 0. 3. 

In [32]:
pd.DataFrame(predictions).to_csv("/kaggle/working/predictions.csv",  index=None)

In [52]:

predictions = "/kaggle/working/predictions.csv"
predictions = np.genfromtxt(predictions, delimiter = '\n')
predictions = np.delete(predictions,0)
predictions = predictions.reshape(5250,1)
print(predictions.shape)


id = '/kaggle/input/uofu-ml-fall-2020/project_data/data/eval.ids'
example = np.genfromtxt(id, delimiter = '\n')
example = example.reshape(5250,1)

print(example.shape)
print(len(predictions))
#output = np.concatenate((example, predictions), axis = 0).astype(int)

output = np.concatenate((example, predictions), axis = 1).astype(int)

print(output.shape)
pd.DataFrame(output).to_csv("/kaggle/working/bow.eval.decisionTree.prediction.csv", header=["example_id", "label"], index=None)

(5250, 1)
(5250, 1)
5250
(5250, 2)
